In [103]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, InputLayer, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import os
from supabase import create_client
from dotenv import load_dotenv

In [104]:
load_dotenv()

True

In [105]:
supabase = create_client(os.getenv('SUPABASE_URL'), os.getenv('SUPABASE_KEY'))

In [106]:
features = ['temp', 'humidity', 'precip', 'windspeed']
cities = ['Caloocan', 'Las Piñas', 'Makati', 'Malabon', 'Mandaluyong', 
          'Manila', 'Marikina', 'Muntinlupa', 'Navotas', 'Parañaque',
          'Pasay', 'Pasig', 'Quezon', 'San Juan', 'Taguig', 'Valenzuela']

In [107]:
def get_table_name(city):
    """Convert city name to table name format"""
    city = city.lower().replace(' ', '_').replace('ñ', 'n')
    if city == "las_piñas": city = "las_pinas"
    if city == "marikina": city = "markina"
    if city == "parañaque": city = "paramaque"
    return f"{city}_city_weather"

In [108]:
def fetch_city_data(city):
    """Fetch weather data from Supabase for a city"""
    table_name = get_table_name(city)
    response = supabase.table(table_name).select("*").execute()
    df = pd.DataFrame(response.data)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    return df[features].copy().ffill()

In [109]:
def create_sequences(data, window_size=21, forecast_size=7):
    """Create training sequences from time series data"""
    X, y = [], []
    for i in range(len(data) - window_size - forecast_size + 1):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size:i + window_size + forecast_size])
    return np.array(X), np.array(y)

In [110]:
def build_model(input_shape):
    """Build LSTM model architecture"""
    model = Sequential([
        InputLayer(input_shape),
        LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.2),
        LSTM(64, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(4 * 7)  # 4 features * 3 days
    ])
    optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

In [111]:
def train_model(city, df):
    """Train model for a specific city"""
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df)
    
    X, y = create_sequences(scaled_data)
    X = X.reshape((X.shape[0], X.shape[1], len(features)))
    y = y.reshape(y.shape[0], -1)
    
    model = build_model((X.shape[1], X.shape[2]))
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, min_delta=0.001),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5),
        ModelCheckpoint(f'weatherModels/{city}_best_model.keras', save_best_only=True)
    ]
    
    history = model.fit(
        X, y,
        epochs=100,
        batch_size=32,
        validation_split=0.1,
        callbacks=callbacks,
        verbose=0
    )
    
    return model, scaler

In [112]:
def predict_future(model, data, scaler, window_size=21, forecast_days=7):
    """Generate future predictions"""
    last_window = scaler.transform(data[-window_size:])
    X_pred = last_window.reshape(1, window_size, len(features))
    pred = model.predict(X_pred)[0]
    pred = pred.reshape(forecast_days, len(features))
    return scaler.inverse_transform(pred)

In [113]:
def save_forecast_to_supabase(city, forecast_df):
    """Save forecast back to Supabase"""
    table_name = f"{get_table_name(city).replace('_weather', '_forecast')}"
    
    # Convert datetime to ISO format strings
    forecast_df = forecast_df.copy()
    forecast_df['datetime'] = pd.to_datetime(forecast_df['datetime'])
    forecast_df['datetime'] = forecast_df['datetime'].dt.strftime('%Y-%m-%d')
    
    records = forecast_df.to_dict('records')
    
    try:
        # Delete old forecasts for these dates
        dates = forecast_df['datetime'].tolist()
        supabase.table(table_name).delete().in_('datetime', dates).execute()
        
        # Insert new forecasts
        response = supabase.table(table_name).upsert(records).execute()
        return response
    except Exception as e:
        print(f"Supabase save error for {city}: {str(e)}")
        return None

In [114]:
def process_city(city):
    """Full processing pipeline for a city"""
    print(f"\nProcessing {city}...")
    
    try:
        # 1. Fetch and validate data
        df = fetch_city_data(city)
        if len(df) < 50:
            print(f"⚠ Not enough data for {city} (only {len(df)} records)")
            return None
            
        # 2. Train model
        model, scaler = train_model(city, df)
        
        # 3. Make predictions
        forecast_values = predict_future(model, df.values, scaler)
        
        # Get current date
        today = pd.Timestamp.now().normalize()
        
        # 4. Create date range (today + next 7 days)
        forecast_dates = pd.date_range(
            start=today,
            periods=8  # Today + 7 days
        )
        
        # Get today's actual weather (last available data)
        today_weather = df.iloc[-1][features].values if not df.empty else [np.nan]*len(features)
        
        # Combine today's actual with 7-day forecast
        all_values = np.vstack([today_weather, forecast_values[:7]])
        
        # Create DataFrame
        forecast_df = pd.DataFrame(
            all_values,
            columns=features,
            index=forecast_dates
        ).reset_index()
        
        forecast_df.insert(0, 'name', f"{city} City, National Capital Region, Philippines")
        forecast_df.rename(columns={'index': 'datetime'}, inplace=True)
        
        # Convert datetime to string format
        forecast_df['datetime'] = forecast_df['datetime'].dt.strftime('%Y-%m-%d')
        
        # 5. Save to Supabase
        if save_forecast_to_supabase(city, forecast_df):
            print(f"✓ {city}: Weather data saved successfully")
            print("\nWeather Report:")
            print(forecast_df[['datetime'] + features].to_string(index=False))
            return forecast_df
        return None
        
    except Exception as e:
        print(f"✗ Error processing {city}: {str(e)}")
        return None

In [115]:
def main():
    os.makedirs("weatherModels", exist_ok=True)
    
    all_forecasts = []
    for city in cities:
        forecast = process_city(city)
        if forecast is not None:
            all_forecasts.append(forecast)
    
    if all_forecasts:
        combined = pd.concat(all_forecasts)
        print("\nAll forecasts completed successfully!")
        print(combined[['name', 'datetime'] + features].to_string(index=False))
    else:
        print("\nNo forecasts were generated")

In [116]:
if __name__ == "__main__":
    main()


Processing Caloocan...
⚠ Not enough data for Caloocan (only 3 records)

Processing Las Piñas...
⚠ Not enough data for Las Piñas (only 1 records)

Processing Makati...
⚠ Not enough data for Makati (only 1 records)

Processing Malabon...
⚠ Not enough data for Malabon (only 1 records)

Processing Mandaluyong...
⚠ Not enough data for Mandaluyong (only 1 records)

Processing Manila...
⚠ Not enough data for Manila (only 1 records)

Processing Marikina...
⚠ Not enough data for Marikina (only 1 records)

Processing Muntinlupa...
⚠ Not enough data for Muntinlupa (only 1 records)

Processing Navotas...
⚠ Not enough data for Navotas (only 1 records)

Processing Parañaque...
⚠ Not enough data for Parañaque (only 1 records)

Processing Pasay...
⚠ Not enough data for Pasay (only 1 records)

Processing Pasig...
⚠ Not enough data for Pasig (only 1 records)

Processing Quezon...


c:\Users\User\Desktop\ForeLastDataTraining\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 698ms/step
✓ Quezon: Weather data saved successfully

Weather Report:
  datetime      temp  humidity    precip  windspeed
2025-04-17 26.300000 91.000000 18.578000  14.300000
2025-04-18 27.658440 89.223633 31.286837  24.943954
2025-04-19 27.575066 87.845558 32.650993  24.683472
2025-04-20 27.694912 88.933441 26.851156  24.946659
2025-04-21 27.547239 88.070374 19.399164  23.088264
2025-04-22 27.615335 87.523300 23.388985  23.950081
2025-04-23 27.586479 88.021523 18.349201  18.582047
2025-04-24 27.651106 87.275391 24.464291  25.399431

Processing San Juan...
⚠ Not enough data for San Juan (only 1 records)

Processing Taguig...
⚠ Not enough data for Taguig (only 2 records)

Processing Valenzuela...
⚠ Not enough data for Valenzuela (only 1 records)

All forecasts completed successfully!
                                             name   datetime      temp  humidity    precip  windspeed
Quezon City, National Capital Region, Philippines 2025-04-17 26.300000 91.000